In [2]:
from pydrake.common import FindResourceOrThrow
from pydrake.geometry import (DrakeVisualizer, SceneGraph)
from pydrake.lcm import DrakeLcm
from pydrake.multibody.plant import MultibodyPlant
from pydrake.multibody.parsing import Parser
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.analysis import Simulator

In [3]:
time_step = 0.01

file_name = FindResourceOrThrow(
        "drake/examples/multibody/cart_pole/cart_pole.sdf")
builder = DiagramBuilder()
scene_graph = builder.AddSystem(SceneGraph())  # records geo info
cart_pole = builder.AddSystem(MultibodyPlant(time_step=time_step)) # important 
cart_pole.RegisterAsSourceForSceneGraph(scene_graph)
Parser(plant=cart_pole).AddModelFromFile(file_name)
cart_pole.Finalize()
assert cart_pole.geometry_source_is_registered()

builder.Connect(
    scene_graph.get_query_output_port(),
    cart_pole.get_geometry_query_input_port())
builder.Connect(
    cart_pole.get_geometry_poses_output_port(),
    scene_graph.get_source_pose_port(cart_pole.get_source_id()))

DrakeVisualizer.AddToBuilder(builder=builder, scene_graph=scene_graph)
diagram = builder.Build()

diagram_context = diagram.CreateDefaultContext()
cart_pole_context = diagram.GetMutableSubsystemContext(
    cart_pole, diagram_context)

cart_pole.get_actuation_input_port().FixValue(cart_pole_context, 0)

cart_slider = cart_pole.GetJointByName("CartSlider")
pole_pin = cart_pole.GetJointByName("PolePin")
cart_slider.set_translation(context=cart_pole_context, translation=0.)
pole_pin.set_angle(context=cart_pole_context, angle=1.)

target_realtime_rate = 1
simulation_time = 5

simulator = Simulator(diagram, diagram_context)
simulator.set_publish_every_time_step(False)
simulator.set_target_realtime_rate(target_realtime_rate) # .1 for fast system, 
simulator.Initialize()
simulator.AdvanceTo(simulation_time)

In [10]:
a = cart_pole.CalcMassMatrixViaInverseDynamics(cart_pole_context)

In [13]:
x = cart_pole.get_frame(10)

TypeError: get_frame(): incompatible function arguments. The following argument types are supported:
    1. (self: pydrake.multibody.plant.MultibodyPlant_[float], frame_index: pydrake.multibody.tree.FrameIndex) -> pydrake.multibody.tree.Frame_[float]

Invoked with: <pydrake.multibody.plant.MultibodyPlant_[float] object at 0x7faa57af95f0>, 10

In [12]:
b = cart_pole.CalcPointsPositions(cart_pole_context)

TypeError: CalcPointsPositions(): incompatible function arguments. The following argument types are supported:
    1. (self: pydrake.multibody.plant.MultibodyPlant_[float], context: pydrake.systems.framework.Context_[float], frame_B: pydrake.multibody.tree.Frame_[float], p_BQi: numpy.ndarray[numpy.float64[m, n], flags.f_contiguous], frame_A: pydrake.multibody.tree.Frame_[float]) -> numpy.ndarray[numpy.float64[m, n]]

Invoked with: <pydrake.multibody.plant.MultibodyPlant_[float] object at 0x7faa57af95f0>, <pydrake.systems.framework.LeafContext_[float] object at 0x7faa57af9670>